# Combinar conjuntos de datos: Merge y join

Una característica esencial que ofrece Pandas son sus operaciones join y merge de alto rendimiento en memoria.

En bases de datos es muy habitual trabajar con este tipo de operaciones con los datos.

La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo puede funcionar en la práctica.

``pd.join`` es muy similar, solo que centrada en índices.

 Puedes ver [aquí](https://pandas.pydata.org/docs/user_guide/merging.html) una comparativa entre distintas funciones/métodos para unir objetos pandas 

Por conveniencia, empezaremos redefiniendo la funcionalidad ``display()`` de la sección anterior:

In [2]:
import pandas as pd
import numpy as np

class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Álgebra Relacional

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para manipular datos relacionales, y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.

El punto fuerte del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los bloques de construcción de operaciones más complicadas sobre cualquier conjunto de datos.

Con este léxico de operaciones fundamentales implementado de forma eficiente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.
Como veremos, estos métodos permiten enlazar datos de diferentes fuentes.

## Categorías de uniones

La función ``pd.merge()`` implementa varios tipos de uniones: las uniones *uno-a-uno*, *many-a-uno* y *many-a-muchos*.
A los tres tipos de uniones se accede mediante una llamada idéntica a la interfaz ``pd.merge()``; el tipo de unión realizada depende de la forma de los datos de entrada.

Aquí mostraremos ejemplos sencillos de los tres tipos de uniones, y discutiremos las opciones detalladas más adelante.

### Uniones uno a uno

Quizá el tipo más sencillo de expresión de fusión sea la unión uno a uno, que en muchos aspectos es muy similar a la concatenación por columnas vista en [Concat](6_Concat-And-Append.ipynb).

Como ejemplo concreto, consideremos los dos ``DataFrames`` siguientes, que contienen información sobre varios empleados de una empresa:

In [3]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un único ``DataFrame``, podemos utilizar la función ``pd.merge()``:

In [4]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [5]:
# El orden de los argumentos determina el orden de las columnas
df3 = pd.merge(df2, df1)
df3

,employee,hire_date,group
0,Lisa,2004,Engineering
1,Bob,2008,Accounting
2,Jake,2012,Engineering
3,Sue,2014,HR


In [6]:
# Existe el método equivalente en un dataframe 
df3 = df2.merge(df1)
df3

,employee,hire_date,group
0,Lisa,2004,Engineering
1,Bob,2008,Accounting
2,Jake,2012,Engineering
3,Sue,2014,HR


In [7]:
df3 = pd.merge(df1, df2, on=['employee'])
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [8]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'empleado': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,empleado,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [9]:
# Ahora no encuentra colunmas en común!
# Quitar comentarios, ejecutar y ver que falla y entender
#df3 = pd.merge(df1, df2)
#df3

In [10]:
# Especificamos las columnas pero... se duplica!
df3 = pd.merge(df1, df2, left_on=['employee'], right_on=["empleado"])
df3

,employee,group,empleado,hire_date
0,Bob,Accounting,Bob,2008
1,Jake,Engineering,Jake,2012
2,Lisa,Engineering,Lisa,2004
3,Sue,HR,Sue,2014


In [11]:
# Cambio de nombre

df1.columns = ['empleado', 'group']
display('df1', 'df2')

,empleado,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,empleado,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [12]:
df3 = pd.merge(df1, df2, on=["empleado"])
df3

,empleado,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna "empleado", y une automáticamente utilizando esta columna como clave.

El resultado de la fusión es un nuevo ``DataFrame`` que combina la información de las dos entradas.
Observa que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna "empleado" difiere entre ``df1`` y ``df2``, y la función ``pd.merge()`` lo tiene en cuenta correctamente.

Además, hay que tener en cuenta que la fusión en general descarta el índice, excepto en el caso especial de las fusiones por índice (véanse las palabras clave ``left_index`` y ``right_index``, de las que hablaremos más adelante).

In [13]:
display('df1', 'df2')

,empleado,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,empleado,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [14]:
# En este caso no tiene sentido
df3 = pd.merge(df1, df2, right_index=True, left_index=True)
df3

,empleado_x,group,empleado_y,hire_date
0,Bob,Accounting,Lisa,2004
1,Jake,Engineering,Bob,2008
2,Lisa,Engineering,Jake,2012
3,Sue,HR,Sue,2014


In [15]:
# Si establecemos indices es más útil
df1.set_index("empleado", inplace=True)
df2 = df2.set_index("empleado")
#

In [16]:
display('df1', 'df2')

,group
empleado,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
empleado,
Lisa,2004
Bob,2008
Jake,2012


In [17]:
df3 = pd.merge(df1, df2, right_index=True, left_index=True)
df3

,group,hire_date
empleado,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


### Many-to-one joins

Las uniones múltiples son uniones en las que una de las dos columnas clave contiene entradas duplicadas.
En este caso, el ``DataFrame`` resultante conservará las entradas duplicadas.
Considere el siguiente ejemplo de unión muchos-a-uno:

In [18]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [19]:
df3 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df4 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR',],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group supervisor
0      Bob   Accounting      Carly
1     Jake  Engineering      Guido
2     Lisa  Engineering      Guido
3      Sue           HR      Steve

El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en uno o más lugares según lo requieran las entradas.

Siguiendo el ejemplo del Banco Pandas 

In [20]:
# Parámetro nrows: solo lee tantas filas como le indiques, para evitar cargar archivos enormes!
oper = pd.read_csv("data/mini_operaciones_1.csv", nrows=500)
oper.head()

,valor,moneda,timestamp,código,id_cliente
0,-17.266329,euros,2024-03-08 05:55:43.110176,100.0,2504900
1,-728.572706,euros,2024-02-12 08:17:28.110176,450.0,26234311
2,-25.142671,euros,2024-03-05 06:31:09.110176,100.0,24009116
3,-944.036110,euros,2024-03-08 01:15:27.110176,250.0,2545272
4,1413.803532,euros,2024-02-15 06:50:12.110176,450.0,17555511


In [21]:
cod = pd.read_csv("data/codigos_operaciones.csv",sep="|")
cod.head()

,código,operación
0,50,transferencia
1,100,domicialización
2,150,nomina
3,200,débito
4,250,crédito


In [22]:
result = pd.merge(oper,cod, on=["código"])
result.head()

,valor,moneda,timestamp,código,id_cliente,operación
0,-17.266329,euros,2024-03-08 05:55:43.110176,100.0,2504900,domicialización
1,-25.142671,euros,2024-03-05 06:31:09.110176,100.0,24009116,domicialización
2,-39.661455,euros,2024-03-03 17:52:32.110176,100.0,1941048,domicialización
3,-8.212729,euros,2024-01-23 21:25:49.110176,100.0,3871180,domicialización
4,-13.803213,euros,2024-03-08 17:26:24.110176,100.0,10556783,domicialización


### Many-to-many joins

Las uniones muchos-a-muchos son un poco confusas conceptualmente, pero sin embargo están bien definidas.
Si la columna clave tanto de la matriz izquierda como de la derecha contiene duplicados, el resultado es una fusión de muchos a muchos.

Quizá esto quede más claro con un ejemplo concreto.

Considere lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.

Realizando una unión de muchos a muchos, podemos recuperar las habilidades asociadas a cualquier persona individual:

In [23]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


In [24]:
# Cuidado, a veces se producen duplicados indeseados

In [25]:
result = pd.merge(oper,cod, on=["código"])
result.head()

,valor,moneda,timestamp,código,id_cliente,operación
0,-17.266329,euros,2024-03-08 05:55:43.110176,100.0,2504900,domicialización
1,-25.142671,euros,2024-03-05 06:31:09.110176,100.0,24009116,domicialización
2,-39.661455,euros,2024-03-03 17:52:32.110176,100.0,1941048,domicialización
3,-8.212729,euros,2024-01-23 21:25:49.110176,100.0,3871180,domicialización
4,-13.803213,euros,2024-03-08 17:26:24.110176,100.0,10556783,domicialización


In [26]:
subcod = pd.read_csv("data/subcodigo.csv", sep=";")
subcod.head()

,operación,tipo
0,crédito,visa
1,crédito,mastercard
2,débito,visa
3,débito,mastercard


In [27]:
result = pd.merge(oper,cod, on=["código"])
result.head()

,valor,moneda,timestamp,código,id_cliente,operación
0,-17.266329,euros,2024-03-08 05:55:43.110176,100.0,2504900,domicialización
1,-25.142671,euros,2024-03-05 06:31:09.110176,100.0,24009116,domicialización
2,-39.661455,euros,2024-03-03 17:52:32.110176,100.0,1941048,domicialización
3,-8.212729,euros,2024-01-23 21:25:49.110176,100.0,3871180,domicialización
4,-13.803213,euros,2024-03-08 17:26:24.110176,100.0,10556783,domicialización


In [28]:
# Duplicados!
result = pd.merge(result,subcod, on=["operación"])
result.head()

,valor,moneda,timestamp,código,id_cliente,operación,tipo
0,-944.036110,euros,2024-03-08 01:15:27.110176,250.0,2545272,crédito,visa
1,-944.036110,euros,2024-03-08 01:15:27.110176,250.0,2545272,crédito,mastercard
2,-75.981546,euros,2024-03-11 10:47:04.110176,250.0,6558141,crédito,visa
3,-75.981546,euros,2024-03-11 10:47:04.110176,250.0,6558141,crédito,mastercard
4,-2853.339559,euros,2024-02-01 20:40:35.110176,250.0,8224745,crédito,visa


Estos tres tipos de uniones se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades.

Pero en la práctica, los conjuntos de datos raramente están tan limpios como con el que estamos trabajando aquí.

En la siguiente sección consideraremos algunas de las opciones proporcionadas por ``pd.merge()`` que permiten ajustar el funcionamiento de las operaciones de unión.

## Especificación de la clave de fusión

Ya hemos visto el comportamiento por defecto de ``pd.merge()``: busca uno o más nombres de columna que coincidan entre las dos entradas y los utiliza como clave.

Sin embargo, a menudo los nombres de las columnas no coinciden tan bien, y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

### La palabra clave ``on``

Lo más sencillo es especificar explícitamente el nombre de la columna clave mediante la palabra clave ``on``, que toma un nombre de columna o una lista de nombres de columna:

In [29]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [30]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [31]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción sólo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

### Las palabras clave ``left_on`` y ``right_on``

En ocasiones, es posible que desee combinar dos conjuntos de datos con nombres de columna diferentes; por ejemplo, podemos tener un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado".
En este caso, podemos utilizar las palabras clave ``left_on`` y ``right_on`` para especificar los dos nombres de columna:

In [32]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, utilizando el método ``drop()`` de ``DataFrame``s:

In [33]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Las palabras clave ``left_index`` y ``right_index``

A veces, en lugar de fusionar en una columna, se desea hacerlo en un índice.
Por ejemplo, sus datos podrían tener este aspecto:

In [34]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Puedes utilizar el índice como clave para la fusión especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [35]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Por comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza una fusión que por defecto une los índices:

In [36]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Si quieres mezclar índices y columnas, puedes combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [37]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; la interfaz para este comportamiento es muy intuitiva.
Para más información sobre esto, consulte la sección ["Merge, Join, and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

## Especificación de la aritmética de conjuntos para uniones


![imagen](./img/merges.png)

En todos los ejemplos anteriores hemos pasado por alto una consideración importante a la hora de realizar una unión: el tipo de aritmética de conjuntos utilizado en la unión.
Esto ocurre cuando un valor aparece en una columna clave pero no en la otra. Veamos este ejemplo:

In [38]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul','Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos fusionado dos conjuntos de datos que sólo tienen una entrada de "nombre" en común: María.
Por defecto, el resultado contiene la *intersección* de los dos conjuntos de entradas; esto es lo que se conoce como *inner join*.

Podemos especificarlo explícitamente mediante la palabra clave ``how``, que por defecto es ``"inner"``:

In [39]:
pd.merge(df6, df7, how='inner', indicator=True)

,name,food,drink,_merge
0,Mary,bread,wine,both


Otras opciones para la palabra clave ``how`` son ``'outer``, ``'left`` y ``'right``.
Un *outer join* devuelve un join sobre la unión de las columnas de entrada, y rellena todos los valores perdidos con NAs:

In [40]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer', indicator=True)")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer', indicator=True)
     name   food drink      _merge
0   Peter   fish   NaN   left_only
1    Paul  beans   NaN   left_only
2    Mary  bread  wine        both
3  Joseph    NaN  beer  right_only

Las opciones *left join* y *right join* devuelven uniones sobre las entradas de la izquierda y la derecha, respectivamente.
Por ejemplo:

In [41]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

In [42]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='right')
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

In [43]:
pd.merge(df6, df7, how='right',indicator=True)

,name,food,drink,_merge
0,Mary,bread,wine,both
1,Joseph,NaN,beer,right_only


Las filas de salida corresponden ahora a las entradas de la entrada izquierda. Utilizando
``how='right'`` funciona de forma similar.

Todas estas opciones pueden aplicarse directamente a cualquiera de los tipos de unión anteriores.

## Nombres de columnas solapados: La palabra clave ``suffixes``

Por último, puede darse el caso de que sus dos ``DataFrame`` de entrada tengan nombres de columna contradictorios.
Considere este ejemplo:

In [44]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Dado que la salida tendría dos nombres de columna en conflicto, la función de fusión añade automáticamente un sufijo ``_x`` o ``_y`` para hacer que las columnas de salida sean únicas.

Si estos valores por defecto no son apropiados, es posible especificar un sufijo personalizado utilizando la palabra clave ``suffixes``:

In [45]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Estos sufijos funcionan en cualquiera de los posibles patrones de unión, y funcionan también si hay varias columnas superpuestas.

Para más información sobre estos patrones, vea [Aggregation and Grouping](8_Aggregation-and-Grouping.ipynb) donde profundizamos un poco más en el álgebra relacional.
Consulte también la documentación de [Pandas "Merge, Join and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) para más información sobre estos temas.

Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

## Ejemplo: Datos de los Estados de EE.UU.

Las operaciones merge y join aparecen con más frecuencia cuando se combinan datos de distintas fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE.UU. y su población.
Los archivos de datos se encuentran en http://github.com/jakevdp/data-USstates/:

In [46]:
# Los siguientes son comandos shell para descargar los datos
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

In [47]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Con esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE.UU. según su densidad de población en 2010.
Es evidente que disponemos de los datos necesarios para obtener este resultado, pero tendremos que combinar los conjuntos de datos para obtenerlo.

Empezaremos con una combinación múltiple que nos dará el nombre completo del estado dentro del ``DataFrame`` de población.

Queremos hacer la combinación basándonos en la columna ``state/region`` de ``pop`` y en la columna ``abbreviation`` de ``abbrevs``.

Utilizaremos ``how='outer'`` para asegurarnos de que no se pierde ningún dato por no coincidir las etiquetas.

In [48]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation', indicator=True)
merged = merged.drop('abbreviation', axis=1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state,_merge
0,AL,under18,2012,1117489.0,Alabama,both
1,AL,total,2012,4817528.0,Alabama,both
2,AL,under18,2010,1130966.0,Alabama,both
3,AL,total,2010,4785570.0,Alabama,both
4,AL,under18,2011,1125763.0,Alabama,both


Comprobemos si se ha producido alguna discordancia, lo que podemos hacer buscando filas con nulos:

In [49]:
# any() busca algún True en una columna
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
_merge          False
dtype: bool

Algunos de los datos de ``población`` son nulos; ¡averigüemos cuáles son!

In [50]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state,_merge
2448,PR,under18,1990,NaN,NaN,left_only
2449,PR,total,1990,NaN,NaN,left_only
2450,PR,total,1991,NaN,NaN,left_only
2451,PR,under18,1991,NaN,NaN,left_only
2452,PR,total,1993,NaN,NaN,left_only


In [51]:
merged[merged['_merge']!="both"]

,state/region,ages,year,population,state,_merge
2448,PR,under18,1990,NaN,NaN,left_only
2449,PR,total,1990,NaN,NaN,left_only
2450,PR,total,1991,NaN,NaN,left_only
2451,PR,under18,1991,NaN,NaN,left_only
2452,PR,total,1993,NaN,NaN,left_only
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,left_only
2540,USA,under18,2011,73902222.0,NaN,left_only
2541,USA,total,2011,311582564.0,NaN,left_only
2542,USA,under18,2012,73708179.0,NaN,left_only


In [52]:
merged.groupby("_merge").count()

C:\Users\jonat\AppData\Local\Temp\ipykernel_17548\1046811352.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.groupby("_merge").count()


,state/region,ages,year,population,state
_merge,,,,,
left_only,96,96,96,76,0
right_only,0,0,0,0,0
both,2448,2448,2448,2448,2448


Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; esto se debe probablemente a que estos datos no están disponibles en la fuente original.

Y lo que es más importante, vemos también que algunas de las nuevas entradas ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [53]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos deducir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (USA), mientras que estas entradas no aparecen en la clave de abreviatura del estado.
Podemos solucionarlo rápidamente introduciendo las entradas adecuadas:

In [54]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
_merge          False
dtype: bool

Se acabaron los nulos en la columna ``state``: ¡listo!

Ahora podemos fusionar el resultado con los datos del área mediante un procedimiento similar.
Al examinar nuestros resultados, querremos unirlos en la columna ``state`` de ambos:

In [55]:
merged.rename(columns={"_merge":"outer"}, inplace=True)
final = pd.merge(merged, areas, on='state', how='left', indicator=True)
final.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
0,AL,under18,2012,1117489.0,Alabama,both,52423.0,both
1,AL,total,2012,4817528.0,Alabama,both,52423.0,both
2,AL,under18,2010,1130966.0,Alabama,both,52423.0,both
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
4,AL,under18,2011,1125763.0,Alabama,both,52423.0,both


Breves notas sobre la igualdad entre dataframes

In [56]:
# NaN != NaN
(final != final).sum()

state/region      0
ages              0
year              0
population       20
state             0
outer             0
area (sq. mi)    48
_merge            0
dtype: int64

In [57]:
# Usamos el metodo equals.
# Más adelante veremos un ejemplo comparando dos dataframes distintos!
final.equals(final)

True

In [58]:
final.groupby('_merge')['_merge'].count()

C:\Users\jonat\AppData\Local\Temp\ipykernel_17548\3929008639.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final.groupby('_merge')['_merge'].count()


_merge
left_only       48
right_only       0
both          2496
Name: _merge, dtype: int64

De nuevo, vamos a comprobar si hay nulos para ver si hay algún desajuste:

In [59]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
outer            False
area (sq. mi)     True
_merge           False
dtype: bool

Hay nulos en la columna ``área``; podemos echar un vistazo para ver qué regiones se ignoraron aquí:

In [60]:
final[final["_merge"]!="both"]

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
2496,USA,under18,1990,64218512.0,United States,left_only,NaN,left_only
2497,USA,total,1990,249622814.0,United States,left_only,NaN,left_only
2498,USA,total,1991,252980942.0,United States,left_only,NaN,left_only
2499,USA,under18,1991,65313018.0,United States,left_only,NaN,left_only
2500,USA,under18,1992,66509177.0,United States,left_only,NaN,left_only
2501,USA,total,1992,256514231.0,United States,left_only,NaN,left_only
2502,USA,total,1993,259918595.0,United States,left_only,NaN,left_only
2503,USA,under18,1993,67594938.0,United States,left_only,NaN,left_only
2504,USA,under18,1994,68640936.0,United States,left_only,NaN,left_only
2505,USA,total,1994,263125826.0,United States,left_only,NaN,left_only


In [61]:
final[final["_merge"]!="both"]['state/region'].unique()

array(['USA'], dtype=object)

Vemos que nuestro ``DataFrame`` de ``areas`` no contiene el área de Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (usando la suma de las áreas de todos los estados, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [62]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
0,AL,under18,2012,1117489.0,Alabama,both,52423.0,both
1,AL,total,2012,4817528.0,Alabama,both,52423.0,both
2,AL,under18,2010,1130966.0,Alabama,both,52423.0,both
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
4,AL,under18,2011,1125763.0,Alabama,both,52423.0,both


Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta que nos interesa, vamos a seleccionar primero la parte de los datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()`` para hacerlo rápidamente (esto requiere tener instalado el paquete ``numexpr``; ver [Pandas de alto rendimiento: ``eval()`` y ``query()``](12_Performance-Eval-and-Query.ipynb)):

In [63]:
data2010 = final[(final["year"] == 2010) & (final["ages"] == 'total')]
data2010.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
91,AK,total,2010,713868.0,Alaska,both,656425.0,both
101,AZ,total,2010,6408790.0,Arizona,both,114006.0,both
189,AR,total,2010,2922280.0,Arkansas,both,53182.0,both
197,CA,total,2010,37333601.0,California,both,163707.0,both


In [64]:
data2010_q = final.query("year == 2010 & ages == 'total'")
data2010_q.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
91,AK,total,2010,713868.0,Alaska,both,656425.0,both
101,AZ,total,2010,6408790.0,Arizona,both,114006.0,both
189,AR,total,2010,2922280.0,Arkansas,both,53182.0,both
197,CA,total,2010,37333601.0,California,both,163707.0,both


In [65]:
data2010.equals(data2010_q)

True

In [66]:
(data2010 != data2010_q).sum()

state/region     0
ages             0
year             0
population       0
state            0
outer            0
area (sq. mi)    0
_merge           0
dtype: int64

In [67]:
# El orden importa en las comparaciones entre dataframes!!
data2010 = data2010.sample(frac=1,replace=False)
#(data2010 != data2010_q).sum()

In [68]:
data2010.equals(data2010_q)

False

Ahora vamos a calcular la densidad de población y mostrarla por orden.
Empezaremos por volver a indexar nuestros datos en el estado y, a continuación, calcularemos el resultado:

In [69]:
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

389     8898.897059
2490    1058.665149
1445    1009.253268
1914     681.339159
293      645.600649
dtype: float64

In [70]:
data2010['density'] = data2010['population'] / data2010['area (sq. mi)']
data2010.sort_values(by=["density"],ascending=False, inplace=True)
data2010.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge,density
389,DC,total,2010,605125.0,District of Columbia,both,68.0,both,8898.897059
2490,PR,total,2010,3721208.0,Puerto Rico,left_only,3515.0,both,1058.665149
1445,NJ,total,2010,8802707.0,New Jersey,both,8722.0,both,1009.253268
1914,RI,total,2010,1052669.0,Rhode Island,both,1545.0,both,681.339159
293,CT,total,2010,3579210.0,Connecticut,both,5544.0,both,645.600649


El resultado es una clasificación de los estados de EE.UU. más Washington, DC, y Puerto Rico por orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington, DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos comprobar el final de la lista:

In [71]:
density.tail()

2010    10.583512
1637     9.537565
1253     6.736171
2405     5.768079
91       1.087509
dtype: float64

Vemos que el estado menos denso, con diferencia, es Alaska, con una media de poco más de un residente por milla cuadrada.

Este tipo de fusión de datos desordenados es una tarea habitual cuando se intenta responder a preguntas utilizando fuentes de datos del mundo real.
Espero que este ejemplo te haya dado una idea de las formas en que puedes combinar las herramientas que hemos visto para obtener información de tus datos.